In [1]:
import numpy as np

In [63]:
def just_f(t):
    global number_of_function_calls
    number_of_function_calls += 1
    #return (t[0]-1)**2 + (t[1]-1)**2                                # Простейшая квадратическая функция
    #return (t[0] - 13) ** 2 - t[0] * t[1] + 3 * t[1] ** 2           # Квадратическая функция на две переменные
    #return 2*t[0]**2+t[0]*t[1]+t[1]*t[2]+2*t[1]**2+3*t[2]**2        # Квадратическая функция на три переменные
    #return (10 * (t[0] - t[1]) ** 2 + (t[0] - 1) ** 2) ** 0.25      # Корневая функция
    return 100*(t[0]**2-t[1])**2+(t[0]-1)**2                        # Функция Розенброка

In [3]:
def grad(t, h):
    
    count = number_of_function_calls
    
    gradient = []
    variable = np.array([])
    size = np.size(t)
    
    for i in range(size):
        variable = np.append(variable, [float(t[i])])
        
    for i in range(size):
        temp = variable[i]
        
        variable[i] += h
        f_1 = just_f(variable)
        
        variable[i] -= 2*h
        f_2 = just_f(variable)
        
        gradient.append(round((f_1-f_2)/(2*h),6))
        variable[i] = temp

    gradient = np.array(gradient)
    
    print("Gradient, number of function calls    :  "+str(number_of_function_calls-count))
    
    return gradient

In [4]:
def svenn(x, S):

    count = number_of_function_calls
    
    def f(lambdq):
        t = x - lambdq * S
        return just_f(t)
    
    lamda = [0]
    
    flag = True

    delta = 0.1 * np.linalg.norm(x) / np.linalg.norm(S)
    
    f_ = [f(lamda[0]-delta)]
    f_.append(f(lamda[0]))
    f_.append(f(lamda[0]+delta))
    
    if f_[0] >= f_[1] <= f_[2]:
        a = lamda[0] - delta
        b = lamda[0] + delta
        
        print("Svenn, number of function calls       :  "+str(number_of_function_calls-count))
        
        return a, b

    elif f_[0] >= f_[1] >= f_[2]:
        lamda.append(lamda[0] + delta)
        del f_[0]
        
    elif f_[0] <= f_[1] <= f_[2]:
        delta = - delta
        flag = False
        lamda.append(lamda[0] + delta)
        del f_[2]
        f_.reverse()
        
    i = 1
    while f_[i] <= f_[i-1]:
        lamda.append(lamda[i] + (2 ** i) * delta)
        f_.append(f(lamda[i+1]))
        i += 1
    
    lamda.append(lamda[i - 1] + (lamda[i] - lamda[i - 1]) / 2)
    f_.append(f(lamda[i+1]))
    
    f_[len(f_)-2], f_[len(f_)-1] = f_[len(f_)-1], f_[len(f_)-2]
    lamda[len(lamda)-2], lamda[len(lamda)-1] = lamda[len(lamda)-1], lamda[len(lamda)-2]
    
    m = min(f_)
    index = f_.index(m)
    
    if flag:
        a = lamda[index - 1]
        b = lamda[index + 1]
    else:
        a = lamda[index + 1]
        b = lamda[index - 1]
        
    print("Svenn, number of function calls       :  "+str(number_of_function_calls-count))
        
    return a, b

In [5]:
def golden_ratio(x, S, epsilon):
    
    def f(lambdq):
        t = x - lambdq * S
        return just_f(t)
    
    a, b = svenn(x, S)
    
    count = number_of_function_calls
    
    L = b - a
    x1 = a + 0.382 * L
    x2 = a + 0.618 * L
    
    f1 = f(x1)
    f2 = f(x2)
    
    while L >= epsilon:
        if f1 <= f2:
            b = x2
            L = b - a
            x2 = x1
            x1 = a + 0.382 * L
            f2 = f1
            f1 = f(x1)
        elif f1 >= f2:
            a = x1
            L = b - a
            x1 = x2
            x2 = a + 0.618 * L
            f1 = f2
            f2 = f(x2)
        
    lambda_top = (a + b) / 2
    
    print("Golden ratio, number of function calls:  "+str(number_of_function_calls-count))
    
    return lambda_top, f(lambda_top)

In [6]:
def dsk_powell(x, S, epsilon):
    
    def f(lambdq):
        t = x - lambdq * S
        return just_f(t)
    
    a, b = svenn(x, S)
    
    count = number_of_function_calls
    
    x1 = a
    x2 = (a+b)/2
    x3 = b
    
    f1 = f(x1)
    f2 = f(x2)
    f3 = f(x3)
    
    a1 = (f2 - f1) / (x2 - x1)
    a2 = ((f3 - f1) / (x3 - x1) - a1)/(x3 - x2)
    
    x_star = (x1 + x2) / 2 - a1 / (2 * a2)
    f_star = f(x_star)
    
    while abs(x2 - x_star) >= epsilon and abs(f2 - f_star) >= epsilon:
        
        if x2 < x_star and f2 > f_star:
            x1 = x2
            x2 = x_star
            
            f1 = f2
            f2 = f_star
        elif x2 > x_star and f2 > f_star:
            x3 = x2
            x2 = x_star
            
            f3 = f2
            f2 = f_star
        elif x2 < x_star and f2 < f_star:
            x3 = x_star
        elif x2 > x_star and f2 < f_star:
            x1 = x_star
            
        a1 = (f2 - f1) / (x2 - x1)
        a2 = ((f3 - f1) / (x3 - x1) - a1)/(x3 - x2)
        
        x_star = (x1 + x2) / 2 - a1 / (2 * a2)
        f_star = f(x_star)
        
    print("DSK-Powell, number of function calls  :  "+str(number_of_function_calls-count))
    
    return x_star, f_star

In [91]:
def dfp(x, eps, h):
    
    x = np.array([x])
    f = np.array([just_f(x[0])])
    
    print("Primary function call                 :  1")
    
    gradients = grad(x[0], h)
    gradients = np.array([gradients])
    
    z = np.zeros(gradients.shape[1])
    
    if np.array_equal(gradients[0], z):
        return x[0], f[0], 0
    
    print("")
    print("Iteration 1")
    print("===========================================")

    I = np.eye(x.shape[1])
    A = I
    S = np.array([np.dot(A, gradients[0])])

    lamda, func = golden_ratio(x[0], S[0], eps)
    #lamda, func = dsk_powell(x[0], S[0], eps)

    x = np.append(x, [x[0] - lamda * S[0]], axis=0)
    f = np.append(f, [func])
    
    gradients = np.append(gradients, [grad(x[1], h)], axis=0)
    
    if np.array_equal(gradients[1], z):
        return x[1], f[1], 1
    
    print("===========================================")
    print("")
    
    check_function = abs(f[1]-f[0])
    check_x = np.linalg.norm(x[1] - x[0])
    
    k = 0
    while check_function >= eps and check_x >= eps:
        print("Iteration "+str(k+2))
        print("===========================================")
        delta_x = np.array([x[k+1] - x[k]])
        delta_g = np.array([gradients[k+1] - gradients[k]])
        
        if lamda < 10 ** (-3):
            A = I
        else:
            A = A + np.dot(delta_x.T, delta_x)/np.dot(delta_x, delta_g.T) - np.dot(np.dot(np.dot(A, delta_g.T), delta_g), A)/np.dot(np.dot(delta_g, A), delta_g.T)
        
        S = np.array([np.dot(A, gradients[k+1])])

        lamda, func = golden_ratio(x[k+1], S[0], eps)
        #lamda, func = dsk_powell(x[k+1], S[0], eps)

        x = np.append(x, [x[k+1] - lamda * S[0]], axis=0)
    
        k+=1
        
        f = np.append(f, [func])  
        gradients = np.append(gradients, [grad(x[k+1], h)], axis=0)
        
        if np.array_equal(gradients[k+1], z):
            return x[k+1], f[k+1], k+1
        
        print("===========================================")
        print("")
        
        check_function = abs(f[k+1]-f[k])
        check_x = np.linalg.norm(x[k+1] - x[k])
    
    k += 1
    
    return x[k], f[k], k

In [116]:
number_of_function_calls = 0
x_min, f_min, number_of_iterations = dfp(np.array([-1.2, 0.]), 0.00001, 0.00001)

Primary function call                 :  1
Gradient, number of function calls    :  4

Iteration 1
Svenn, number of function calls       :  7
Golden ratio, number of function calls:  13
Gradient, number of function calls    :  4

Iteration 2
Svenn, number of function calls       :  3
Golden ratio, number of function calls:  20
Gradient, number of function calls    :  4

Iteration 3
Svenn, number of function calls       :  6
Golden ratio, number of function calls:  22
Gradient, number of function calls    :  4

Iteration 4
Svenn, number of function calls       :  7
Golden ratio, number of function calls:  29
Gradient, number of function calls    :  4

Iteration 5
Svenn, number of function calls       :  6
Golden ratio, number of function calls:  29
Gradient, number of function calls    :  4

Iteration 6
Svenn, number of function calls       :  8
Golden ratio, number of function calls:  29
Gradient, number of function calls    :  4

Iteration 7
Svenn, number of function calls       :  8


In [117]:
number_of_function_calls

756

In [118]:
np.round_(x_min,6)

array([0.999994, 0.999988])

In [ ]:
np.round_(f_min,6)